In [18]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
from tabulate import tabulate
from hyperopt import hp, fmin, tpe, Trials, space_eval
from sklearn.model_selection import GroupKFold, StratifiedKFold
import matplotlib.pyplot as plt
import cvxpy as cvx
from sklearn import metrics
from sklearn.metrics.pairwise import manhattan_distances
import quadprog
from sklearn.preprocessing import LabelEncoder
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB

Function Library

In [19]:
# calculate CC/ACC accuary score
def Accuary(estimate, actual):
  return 1 - (abs(estimate - actual) / actual)

# calculate train_score, test_score
def getScores(X_train, X_test, Y_train, nclasses):

    clf1 = LinearDiscriminantAnalysis()
    clf2 = RandomForestClassifier(n_estimators=100)
    clf3 = SVC(probability=True)

    # 使用 Platt Scaling 校准概率分数
    clf1_calibrated = CalibratedClassifierCV(clf1, method='sigmoid', cv=5)
    clf2_calibrated = CalibratedClassifierCV(clf2, method='sigmoid', cv=5)
    clf3_calibrated = CalibratedClassifierCV(clf3, method='sigmoid', cv=5)

    model = VotingClassifier(estimators=[
        ('lda', clf1_calibrated), ('rf', clf2_calibrated), ('svc', clf3_calibrated)], voting='soft')

    train_scores = np.zeros((len(X_train), nclasses))
    test_scores = np.zeros((len(X_test), nclasses))

    Y_cts = np.unique(Y_train, return_counts=True)
    nfolds = min(10, min(Y_cts[1]))
    
    if nfolds > 1:
        kfold = model_selection.StratifiedKFold(n_splits=nfolds, random_state=1, shuffle=True)
        for train_idx, test_idx in kfold.split(X_train, Y_train):
            model.fit(X_train[train_idx], Y_train[train_idx])
            train_scores[test_idx] = model.predict_proba(X_train[test_idx])

    # 训练最终模型并预测
    model.fit(X_train, Y_train)
    test_scores = model.predict_proba(X_test)
            
    return train_scores, test_scores

# EMQ function
def EMQ(test_scores, nclasses):
    max_it = 1000        # Max num of iterations
    eps = 1e-1           # Small constant for stopping criterium

    p_tr = [0.25, 0.25, 0.25, 0.25]
    p_s = np.copy(p_tr)
    p_cond_tr = np.array(test_scores)
    p_cond_s = np.zeros(p_cond_tr.shape)
    prob_arrays = []

    for _ in range(max_it):
        # Add Laplacian smoothing
        # r = (p_s + alpha) / (p_tr + (alpha * nclasses))
        r = p_s / p_tr
        
        p_cond_s = p_cond_tr * r
        s = np.sum(p_cond_s, axis = 1)
        for c in range(nclasses):
            p_cond_s[:,c] = p_cond_s[:,c] / s

        prob_arrays.append(p_cond_s)
        p_s_old = np.copy(p_s)
        p_s = np.sum(p_cond_s, axis = 0) / p_cond_s.shape[0]
        if (np.sum(np.abs(p_s - p_s_old)) < eps):
            break

    return (p_s/np.sum(p_s))

def GAC(train_scores, test_scores, train_labels, nclasses):
   
    yt_hat = np.argmax(train_scores, axis = 1)
    y_hat = np.argmax(test_scores, axis = 1)
    CM = metrics.confusion_matrix(train_labels, yt_hat, normalize="true").T
    p_y_hat = np.zeros(nclasses)
    values, counts = np.unique(y_hat, return_counts=True)
    p_y_hat[values] = counts 
    p_y_hat = p_y_hat/p_y_hat.sum()
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value

def GPAC(train_scores, test_scores, train_labels, nclasses):

    CM = np.zeros((nclasses, nclasses))
    for i in range(nclasses):
        idx = np.where(train_labels == i)[0]
        CM[i] = np.sum(train_scores[idx], axis=0)
        CM[i] /= np.sum(CM[i])
    CM = CM.T
    p_y_hat = np.sum(test_scores, axis = 0)
    p_y_hat = p_y_hat / np.sum(p_y_hat)
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value

def FM(train_scores, test_scores, train_labels, nclasses):

    CM = np.zeros((nclasses, nclasses))
    y_cts = np.array([np.count_nonzero(train_labels == i) for i in range(nclasses)])
    p_yt = y_cts / train_labels.shape[0]
    for i in range(nclasses):
        idx = np.where(train_labels == i)[0]
        CM[:, i] += np.sum(train_scores[idx] > p_yt, axis=0) 
    CM = CM / y_cts
    p_y_hat = np.sum(test_scores > p_yt, axis = 0) / test_scores.shape[0]
    
    p_hat = cvx.Variable(CM.shape[1])
    constraints = [p_hat >= 0, cvx.sum(p_hat) == 1.0]
    problem = cvx.Problem(cvx.Minimize(cvx.norm(CM @ p_hat - p_y_hat)), constraints)
    problem.solve()
    return p_hat.value


Data Preparation

In [20]:
# Load datasets
train_incubator = pd.read_csv('train_incubator.csv')
test_sf2 = pd.read_csv('test_sf2.csv')

# Check number of examples per class
print (train_incubator['class'].value_counts())
print (test_sf2['class'].value_counts())

class
arabiensis_female    3000
culex_female         3000
funestus_female      3000
gambiae_female       3000
Name: count, dtype: int64
class
gambiae_female       600
culex_female         522
funestus_female      512
arabiensis_female    428
Name: count, dtype: int64


In [21]:
# Define feature sets
special_features = ['temperature', 'duration', 'humidity']
wbf_features = ['L_harmcherry_wbf_mean','L_harmcherry_wbf_stddev']
freq_features = [f'L_harmcherry_h{i}_freq' for i in range(1,9)]
basefreq_features = [f'L_harmcherry_h{i}_basefreq' for i in range(1,9)]
relbasefreq_features = [f'L_harmcherry_h{i}_relbasefreq' for i in range(1,9)]
power_features = [f'L_harmcherry_h{i}_power' for i in range(1,9)]
relpower_features = [f'L_harmcherry_h{i}_relpower' for i in range(1,9)]
invented_features = [f'L_harmcherry_h{i}_invented' for i in range(1,9)]

feature_set = special_features+wbf_features+freq_features+basefreq_features+relbasefreq_features+power_features

In [22]:
X_train = pd.DataFrame(train_incubator, columns=feature_set).to_numpy()
y_train = train_incubator['class'].values 

X_test = pd.DataFrame(test_sf2, columns=feature_set).to_numpy()
y_test = test_sf2['class'].values

nclasses = len(train_incubator['class'].unique())

Ensemble model, voting method：
1. LDA  2. RFC  3. SVC

In [23]:
clf1 = LinearDiscriminantAnalysis()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = SVC(probability=True)
clf4 = GaussianNB()

 # 使用 Platt Scaling 校准概率分数
clf1_calibrated = CalibratedClassifierCV(clf1, method='sigmoid', cv=5)
clf2_calibrated = CalibratedClassifierCV(clf2, method='sigmoid', cv=5)
clf3_calibrated = CalibratedClassifierCV(clf3, method='sigmoid', cv=5)
clf4_calibrated = CalibratedClassifierCV(clf4, method='sigmoid', cv=5)

# model = VotingClassifier(estimators=[
#     ('lda', clf1), ('rf', clf2), ('svc', clf3), ('NB', clf4)], voting='soft')
# model.fit(X_train, y_train)

model = VotingClassifier(estimators=[
    ('lda', clf1_calibrated), ('rf', clf2_calibrated), ('svc', clf3_calibrated), ('gnb', clf4_calibrated)], voting='soft')
model.fit(X_train, y_train)

p_labels = model.predict(X_test)
acc = accuracy_score(y_test, p_labels)

print(f"\tAcc: {acc:.4f}")
print(classification_report(y_test, p_labels, labels=np.unique(y_test)))

cf = confusion_matrix(y_test, p_labels, labels=np.unique(y_train))
print(tabulate(cf, headers=np.unique(y_train), tablefmt='fancy_grid'))


	Acc: 0.5126
                   precision    recall  f1-score   support

arabiensis_female       0.34      0.37      0.35       428
     culex_female       0.66      0.59      0.62       522
  funestus_female       0.53      0.91      0.67       512
   gambiae_female       0.52      0.20      0.29       600

         accuracy                           0.51      2062
        macro avg       0.51      0.52      0.48      2062
     weighted avg       0.52      0.51      0.48      2062

╒═════════════════════╤════════════════╤═══════════════════╤══════════════════╕
│   arabiensis_female │   culex_female │   funestus_female │   gambiae_female │
╞═════════════════════╪════════════════╪═══════════════════╪══════════════════╡
│                 159 │             87 │               112 │               70 │
├─────────────────────┼────────────────┼───────────────────┼──────────────────┤
│                 173 │            308 │                18 │               23 │
├─────────────────────┼─────────

Classify and Count(CC)

In [24]:
arabiensis_CC_estimate = cf[0][0]+cf[1][0]+cf[2][0]+cf[3][0]
arabiensis_actual = cf[0][0]+cf[0][1]+cf[0][2]+cf[0][3]
arabiensis_CC = Accuary(arabiensis_CC_estimate, arabiensis_actual)
print('arabiensis CC:', arabiensis_CC)

culex_CC_estimate = cf[0][1]+cf[1][1]+cf[2][1]+cf[3][1]
culex_actual = cf[1][0]+cf[1][1]+cf[1][2]+cf[1][3]
culex_CC = Accuary(culex_CC_estimate, culex_actual)
print('culex CC:', culex_CC)

funestus_CC_estimate = cf[0][2]+cf[1][2]+cf[2][2]+cf[3][2]
funestus_actual = cf[2][0]+cf[2][1]+cf[2][2]+cf[2][3]
funestus_CC = Accuary(funestus_CC_estimate, funestus_actual)
print('funestus CC:', funestus_CC)

gambiae_CC_estimate = cf[0][3]+cf[1][3]+cf[2][3]+cf[3][3]
gambiae_actual = cf[3][0]+cf[3][1]+cf[3][2]+cf[3][3]
gambiae_CC = Accuary(gambiae_CC_estimate, gambiae_actual)
print('gambiae CC:', gambiae_CC)


arabiensis CC: 0.8948598130841121
culex CC: 0.8946360153256705
funestus CC: 0.26953125
gambiae CC: 0.3933333333333333


Adjusted Classify and Count(ACC)

In [25]:
# class's tpr
arabiensis_estimate_number = cf[0][0]
culex_estimate_number = cf[1][1]
funestus_estimate_number = cf[2][2]
gambiae_estimate_number = cf[3][3]

arabiensis_semi_tpr = arabiensis_estimate_number / arabiensis_actual
print("arabiensis's TPR at semi_field:", arabiensis_semi_tpr)

culex_semi_tpr = culex_estimate_number / culex_actual
print("culex's TPR at semi_field:", culex_semi_tpr)

funestus_semi_tpr = funestus_estimate_number / funestus_actual
print("funestus's TPR at semi_field:", funestus_semi_tpr)

gambiae_semi_tpr = gambiae_estimate_number / gambiae_actual
print("gambiae's TPR at semi_field:", gambiae_semi_tpr)

arabiensis's TPR at semi_field: 0.37149532710280375
culex's TPR at semi_field: 0.5900383141762452
funestus's TPR at semi_field: 0.912109375
gambiae's TPR at semi_field: 0.205


In [26]:
X = pd.DataFrame(train_incubator, columns=feature_set).to_numpy()
y = train_incubator['class'].values 

clf1 = LinearDiscriminantAnalysis()
clf2 = RandomForestClassifier(n_estimators=100)
clf3 = SVC(probability=True)
clf4 = GaussianNB()

 # 使用 Platt Scaling 校准概率分数
clf1_calibrated = CalibratedClassifierCV(clf1, method='sigmoid', cv=5)
clf2_calibrated = CalibratedClassifierCV(clf2, method='sigmoid', cv=5)
clf3_calibrated = CalibratedClassifierCV(clf3, method='sigmoid', cv=5)
clf4_calibrated = CalibratedClassifierCV(clf4, method='sigmoid', cv=5)

# model = VotingClassifier(estimators=[
#     ('lda', clf1), ('rf', clf2), ('svc', clf3), ('NB', clf4)], voting='soft')

model = VotingClassifier(estimators=[
    ('lda', clf1_calibrated), ('rf', clf2_calibrated), ('svc', clf3_calibrated), ('gnb', clf4_calibrated)], voting='soft')

groups = train_incubator['sensor'].values
group_kfold = GroupKFold(n_splits=6)

arabiensis_lab_tpr = 0
culex_lab_tpr = 0
funestus_lab_tpr = 0
gambiae_lab_tpr = 0

for train_index_lab, test_index_lab in group_kfold.split(X, y, groups):
  X_train_lab, y_train_lab, X_test_lab, y_test_lab = X[train_index_lab], y[train_index_lab], X[test_index_lab], y[test_index_lab]
  model.fit(X[train_index_lab], y[train_index_lab])

  p_labels_lab = model.predict(X_test_lab)
  a_labels_lab = y_test_lab
  acc = accuracy_score(a_labels_lab, p_labels_lab)
  print('number: ', len(a_labels_lab))

  print("\tAcc: %.4f" % acc)
  print (classification_report(a_labels_lab, p_labels_lab, labels=np.unique(y_test_lab)))
      
  cf_lab = confusion_matrix(a_labels_lab, p_labels_lab, labels=np.unique(y_train_lab))
  arabiensis_actual_lab = cf_lab[0][0]+cf_lab[0][1]+cf_lab[0][2]+cf_lab[0][3]
  culex_actual_lab = cf_lab[1][0]+cf_lab[1][1]+cf_lab[1][2]+cf_lab[1][3]
  funestus_actual_lab = cf_lab[2][0]+cf_lab[2][1]+cf_lab[2][2]+cf_lab[2][3]
  gambiae_actual_lab = cf_lab[3][0]+cf_lab[3][1]+cf_lab[3][2]+cf_lab[3][3]
  arabiensis_lab_tpr += cf_lab[0][0] / arabiensis_actual_lab
  culex_lab_tpr += cf_lab[1][1] / culex_actual_lab
  funestus_lab_tpr += cf_lab[2][2] / funestus_actual_lab
  gambiae_lab_tpr += cf_lab[3][3] / gambiae_actual_lab

  print(tabulate(cf_lab, headers=np.unique(y_train_lab), tablefmt='fancy_grid'))

arabiensis_lab_tpr = arabiensis_lab_tpr / 6
culex_lab_tpr = culex_lab_tpr / 6
funestus_lab_tpr = funestus_lab_tpr / 6
gambiae_lab_tpr = gambiae_lab_tpr / 6

print('arabiensis_lab_tpr:', arabiensis_lab_tpr)
print('culex_lab_tpr:', culex_lab_tpr)
print('funestus_lab_tpr:', funestus_lab_tpr)
print('gambiae_lab_tpr:', gambiae_lab_tpr)


number:  2605
	Acc: 0.4994
                   precision    recall  f1-score   support

arabiensis_female       0.42      0.28      0.34       636
     culex_female       0.40      0.84      0.54       495
  funestus_female       0.62      0.72      0.67       713
   gambiae_female       0.59      0.25      0.35       761

         accuracy                           0.50      2605
        macro avg       0.51      0.52      0.48      2605
     weighted avg       0.52      0.50      0.47      2605

╒═════════════════════╤════════════════╤═══════════════════╤══════════════════╕
│   arabiensis_female │   culex_female │   funestus_female │   gambiae_female │
╞═════════════════════╪════════════════╪═══════════════════╪══════════════════╡
│                 180 │            271 │               107 │               78 │
├─────────────────────┼────────────────┼───────────────────┼──────────────────┤
│                  48 │            416 │                11 │               20 │
├─────────────────

In [27]:
arabiensis_ACC_estimate = arabiensis_estimate_number / arabiensis_lab_tpr
culex_ACC_estimate = culex_estimate_number / culex_lab_tpr
funestus_ACC_estimate = funestus_estimate_number / funestus_lab_tpr
gambiae_ACC_estimate = gambiae_estimate_number / gambiae_lab_tpr

arabiensis_ACC = Accuary(arabiensis_ACC_estimate, arabiensis_actual)
culex_ACC = Accuary(culex_ACC_estimate, culex_actual)
funestus_ACC = Accuary(funestus_ACC_estimate, funestus_actual)
gambiae_ACC = Accuary(gambiae_ACC_estimate, gambiae_actual)

print('arabiensis ACC: ', arabiensis_ACC)
print('culex ACC: ', culex_ACC)
print('funestus ACC: ', funestus_ACC)
print('gambiae ACC: ', gambiae_ACC)

arabiensis ACC:  0.6876202231127856
culex ACC:  0.7953858066059188
funestus ACC:  0.818859523641541
gambiae ACC:  0.6719094857318286


Probabilistic Classify and Count(PCC)

In [28]:
train_scores, test_scores = getScores(X_train, X_test, y_train, nclasses)
estimated_counts = np.mean(test_scores, axis=0) * len(test_scores)

arabiensis_PCC_estimate = estimated_counts[0]
culex_PCC_estimate = estimated_counts[1]
funestus_PCC_estimate = estimated_counts[2]
gambiae_PCC_estimate = estimated_counts[3]

arabiensis_PCC = Accuary(arabiensis_PCC_estimate, arabiensis_actual)
culex_PCC = Accuary(culex_PCC_estimate, culex_actual)
funestus_PCC = Accuary(funestus_PCC_estimate, funestus_actual)
gambiae_PCC = Accuary(gambiae_PCC_estimate, gambiae_actual)

print('arabiensis PCC: ', arabiensis_PCC)
print('culex PCC: ', culex_PCC)
print('funestus PCC: ', funestus_PCC)
print('gambiae PCC: ', gambiae_PCC)

arabiensis PCC:  0.6945410373345047
culex PCC:  0.828766885213478
funestus PCC:  0.9896402763636161
gambiae PCC:  0.9222384523265049


Expectation Maximisation for Quantification(EMQ)

In [29]:
res = EMQ(test_scores, nclasses)

arabiensis_EMQ_estimate = res[0] * len(test_scores)
culex_EMQ_estimate = res[1] * len(test_scores)
funestus_EMQ_estimate = res[2] * len(test_scores)
gambiae_EMQ_estimate = res[3] * len(test_scores)

arabiensis_EMQ = Accuary(arabiensis_EMQ_estimate, arabiensis_actual)
culex_EMQ = Accuary(culex_EMQ_estimate, culex_actual)
funestus_EMQ = Accuary(funestus_EMQ_estimate, funestus_actual)
gambiae_EMQ = Accuary(gambiae_EMQ_estimate, gambiae_actual)

print('arabiensis EMQ: ', arabiensis_EMQ)
print('culex EMQ: ', culex_EMQ)
print('funestus EMQ: ', funestus_EMQ)
print('gambiae EMQ: ', gambiae_EMQ)

arabiensis EMQ:  0.6945410373345041
culex EMQ:  0.8287668852134783
funestus EMQ:  0.9896402763636156
gambiae EMQ:  0.9222384523265053


GAC

In [30]:
# 编码训练标签
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(y_train)

res = GAC(train_scores, test_scores, train_labels, nclasses)

arabiensis_GAC_estimate = res[0] * len(test_scores)
culex_GAC_estimate = res[1] * len(test_scores)
funestus_GAC_estimate = res[2] * len(test_scores)
gambiae_GAC_estimate = res[3] * len(test_scores)

arabiensis_GAC = Accuary(arabiensis_GAC_estimate, arabiensis_actual)
culex_GAC = Accuary(culex_GAC_estimate, culex_actual)
funestus_GAC = Accuary(funestus_GAC_estimate, funestus_actual)
gambiae_GAC = Accuary(gambiae_GAC_estimate, gambiae_actual)

print('arabiensis GAC: ', arabiensis_GAC)
print('culex GAC: ', culex_GAC)
print('funestus GAC: ', culex_GAC)
print('gambiae GAC: ', gambiae_GAC)


arabiensis GAC:  0.9348546236957274
culex GAC:  0.4851076663428754
funestus GAC:  0.4851076663428754
gambiae GAC:  0.4974779301131501


c:\Users\Asus\anaconda3\envs\comp9991\lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


GPAC

In [31]:
res = GPAC(train_scores, test_scores, train_labels, nclasses)

arabiensis_GPAC_estimate = res[0] * len(test_scores)
culex_GPAC_estimate = res[1] * len(test_scores)
funestus_GPAC_estimate = res[2] * len(test_scores)
gambiae_GPAC_estimate = res[3] * len(test_scores)

arabiensis_GPAC = Accuary(arabiensis_GPAC_estimate, arabiensis_actual)
culex_GPAC = Accuary(culex_GPAC_estimate, culex_actual)
funestus_GPAC = Accuary(funestus_GPAC_estimate, funestus_actual)
gambiae_GPAC = Accuary(gambiae_GPAC_estimate, gambiae_actual)

print('arabiensis GPAC: ', arabiensis_GPAC)
print('culex GPAC: ', culex_GPAC)
print('funestus GPAC: ', culex_GPAC)
print('gambiae GPAC: ', gambiae_GPAC)

arabiensis GPAC:  0.3767976947611238
culex GPAC:  0.11523932370097545
funestus GPAC:  0.11523932370097545
gambiae GPAC:  0.5774488654376889


FM

In [32]:
res = FM(train_scores, test_scores, train_labels, nclasses)

arabiensis_FM_estimate = res[0] * len(test_scores)
culex_FM_estimate = res[1] * len(test_scores)
funestus_FM_estimate = res[2] * len(test_scores)
gambiae_FM_estimate = res[3] * len(test_scores)

arabiensis_FM = Accuary(arabiensis_FM_estimate, arabiensis_actual)
culex_FM = Accuary(culex_FM_estimate, culex_actual)
funestus_FM = Accuary(funestus_FM_estimate, funestus_actual)
gambiae_FM = Accuary(gambiae_FM_estimate, gambiae_actual)

print('arabiensis FM: ', arabiensis_FM)
print('culex FM: ', culex_FM)
print('funestus FM: ', culex_FM)
print('gambiae FM: ', gambiae_FM)

arabiensis FM:  0.0050439052891326774
culex FM:  0.10147810322149009
funestus FM:  0.10147810322149009
gambiae FM:  0.8938216627613574


In [33]:
from sklearn.metrics import confusion_matrix, roc_curve

label_encoder = LabelEncoder()
test_labels = label_encoder.fit_transform(y_test)

def find_threshold_x(y_true, y_scores, nclasses):
    thresholds = np.zeros(nclasses)
    for i in range(nclasses):
        fpr, tpr, thr = roc_curve(y_true == i, y_scores[:, i])
        target_fpr = 1 - tpr
        idx = np.argmin(np.abs(fpr - target_fpr))
        thresholds[i] = thr[idx]
    return thresholds

def find_threshold_max(y_true, y_scores, nclasses):
    thresholds = np.zeros(nclasses)
    for i in range(nclasses):
        fpr, tpr, thr = roc_curve(y_true == i, y_scores[:, i])
        diff = tpr - fpr
        idx = np.argmax(diff)
        thresholds[i] = thr[idx] if idx < len(thr) else 0.5
    return thresholds

def find_threshold_t20(y_true, y_scores, nclasses):
    thresholds = np.zeros(nclasses)
    for i in range(nclasses):
        fpr, tpr, thr = roc_curve(y_true == i, y_scores[:, i])
        idx = np.argmin(np.abs(tpr - 0.2))
        thresholds[i] = thr[idx] if idx < len(thr) else 0.2
    return thresholds

def calculate_tpr_fpr(y_true, y_scores, threshold):
    y_pred = (y_scores >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    tpr = tp / (tp + fn)
    fpr = fp / (fp + tn)
    return tpr, fpr

thresholds = find_threshold_t20(test_labels, test_scores, nclasses)

tpr = np.zeros(nclasses)
fpr = np.zeros(nclasses)
for i in range(nclasses):
    tpr[i], fpr[i] = calculate_tpr_fpr(test_labels == i, test_scores[:, i], thresholds[i])
prevalence = np.array([np.mean(test_labels == i) for i in range(nclasses)])
p_hat = (prevalence - fpr) / (tpr - fpr)
p_hat = np.clip(p_hat, 0, 1)
p_hat /= p_hat.sum()

arabiensis_threshold_estimate = p_hat[0] * len(test_scores)
culex_threshold_estimate = p_hat[1] * len(test_scores)
funestus_threshold_estimate = p_hat[2] * len(test_scores)
gambiae_threshold_estimate = p_hat[3] * len(test_scores)

arabiensis_threshold = Accuary(arabiensis_threshold_estimate, arabiensis_actual)
culex_threshold = Accuary(culex_threshold_estimate, culex_actual)
funestus_threshold = Accuary(funestus_threshold_estimate, funestus_actual)
gambiae_threshold = Accuary(gambiae_threshold_estimate, gambiae_actual)

print('arabiensis threshold: ', arabiensis_threshold)
print('culex threshold: ', culex_threshold)
print('funestus threshold: ', culex_threshold)
print('gambiae threshold: ', gambiae_threshold)


arabiensis threshold:  0.7934968735791429
culex threshold:  0.9892401113182506
funestus threshold:  0.9892401113182506
gambiae threshold:  0.8606388968468781
